# Machine Learning Language Translation

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np

batch_size=64 #Batch size for training
epochs=100 #No of epochs to train for
latent_dim=256 #Latent dimensionality of the encoding space
num_samples=10000 #no of samples to train on
#path to the data txt file on disk
data_path=r"C:\Users\ShibuKumar\Downloads\Datas\fra.txt"

In [20]:
#vectorize the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,"r",encoding="utf-8") as f:
    lines=f.read().split("\n")
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text,target_text=line.split("\t")
    #we use "tab" as the "start sequence" character.
    #for the targets, and "\n" as "end sequence" character.
    target_text="\t"+target_text+"\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)       

In [30]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [35]:
print("Number of Samples:",len(input_texts))
print("Number of unique input tokens:",num_encoder_tokens)
print("Number of unique output tokens:",num_decoder_tokens)
print("Max sequence length for inputs:",max_encoder_seq_length)
print("Max sequence length for outputs:",max_decoder_seq_length)

Number of Samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [36]:
input_token_index=dict([
    (char,i) for i,char in enumerate(input_characters)
])
target_token_index=dict([
    (char,i) for i,char in enumerate(target_characters)
])

In [44]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  '%': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '0': 9,
  '1': 10,
  '2': 11,
  '3': 12,
  '4': 13,
  '5': 14,
  '6': 15,
  '7': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A': 21,
  'B': 22,
  'C': 23,
  'D': 24,
  'E': 25,
  'F': 26,
  'G': 27,
  'H': 28,
  'I': 29,
  'J': 30,
  'K': 31,
  'L': 32,
  'M': 33,
  'N': 34,
  'O': 35,
  'P': 36,
  'Q': 37,
  'R': 38,
  'S': 39,
  'T': 40,
  'U': 41,
  'V': 42,
  'W': 43,
  'Y': 44,
  'a': 45,
  'b': 46,
  'c': 47,
  'd': 48,
  'e': 49,
  'f': 50,
  'g': 51,
  'h': 52,
  'i': 53,
  'j': 54,
  'k': 55,
  'l': 56,
  'm': 57,
  'n': 58,
  'o': 59,
  'p': 60,
  'q': 61,
  'r': 62,
  's': 63,
  't': 64,
  'u': 65,
  'v': 66,
  'w': 67,
  'x': 68,
  'y': 69,
  'z': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '3': 15,
  '5': 16,
  '6': 17,
  '8': 18,
  '9': 19,
  ':': 20,
  '?'

In [45]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
    dtype="float32")
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype="float32")
decoder_target_data=np.zeros(
    (len(input_texts,),max_decoder_seq_length,num_decoder_tokens),
    dtype="float32")

In [46]:
#one hot representation
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1.
    encoder_input_data[i,t+1:,input_token_index[" "]]=1.
    for t,char in enumerate(target_text):
        #decoder target data is ahead of decoder_input_data by one time step
        decoder_input_data[i,t,target_token_index[char]]=1.
        if t>0:
            #decoder_target_data will be ahead by one timestep
            #and will not include the start character.
            decoder_target_data[i,t-1,target_token_index[char]]=1.
    decoder_input_data[i,t+1:,target_token_index[" "]]=1.
    decoder_target_data[i,t:,target_token_index[" "]]=1.

In [47]:
encoder_input_data[0].shape

(16, 71)

In [52]:
#Define an input sequence and process it.
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
#we discard "encoder_outputs" and only keep the states
encoder_states=[state_h,state_c]

In [53]:
#set up the decoder,using "encoder_states" as inital state.
decoder_inputs=Input(shape=(None,num_decoder_tokens))
#we set up our decoder to return full output sequence,
#and to return internal states as well. we don"t use the
#return states in the training model, but we will use them in inference.
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation="softmax")
decoder_outputs=decoder_dense(decoder_outputs)

In [54]:
#Define the model that will turn
#"encoder_input_data" &"decoder_input_data" into "decoder_target_data"
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

#Run training
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 61s 8ms/sample - loss: 1.2105 - acc: 0.7208 - val_loss: 1.1300 - val_acc: 0.7012
Epoch 2/100
8000/8000 [==============================] - 59s 7ms/sample - loss: 0.8636 - acc: 0.7661 - val_loss: 0.8896 - val_acc: 0.7560
Epoch 3/100
8000/8000 [==============================] - 57s 7ms/sample - loss: 0.6947 - acc: 0.8058 - val_loss: 0.7582 - val_acc: 0.7788
Epoch 4/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.5972 - acc: 0.8264 - val_loss: 0.6819 - val_acc: 0.7991
Epoch 5/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.5492 - acc: 0.8393 - val_loss: 0.6489 - val_acc: 0.8082
Epoch 6/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.5116 - acc: 0.8493 - val_loss: 0.6121 - val_acc: 0.8176
Epoch 7/100
8000/8000 [=======

8000/8000 [==============================] - 55s 7ms/sample - loss: 0.1024 - acc: 0.9679 - val_loss: 0.6171 - val_acc: 0.8650
Epoch 58/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.0995 - acc: 0.9690 - val_loss: 0.6264 - val_acc: 0.8647
Epoch 59/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.0974 - acc: 0.9695 - val_loss: 0.6240 - val_acc: 0.8650
Epoch 60/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.0958 - acc: 0.9697 - val_loss: 0.6317 - val_acc: 0.8652
Epoch 61/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.0934 - acc: 0.9707 - val_loss: 0.6398 - val_acc: 0.8641
Epoch 62/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.0911 - acc: 0.9715 - val_loss: 0.6434 - val_acc: 0.8643
Epoch 63/100
8000/8000 [==============================] - 55s 7ms/sample - loss: 0.0898 - acc: 0.9717 - val_loss: 0.6454 - val_acc: 0.8650
Epoch 64/100
8000/8000 [================

In [66]:
#Next:inference mode(sampling)
#Here's the drill:
#1) encode input and retrieve initial decoder state
#2) run one step of decoder with this initial state
#and a "start of sequence" taken as target.
#output will be a next target token
#3)Repeat with the current target taken and current states

#Define sampling models
encoder_model=Model(encoder_inputs,encoder_states)

decoder_state_inputs_h=Input(shape=(latent_dim,))
decoder_state_inputs_c=Input(shape=(latent_dim,))
decoder_state_inputs=[decoder_state_inputs_h,decoder_state_inputs_c]
decoder_outputs,state_h,state_c=decoder_lstm(
    decoder_inputs,initial_state=decoder_state_inputs)
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model(
    [decoder_inputs]+decoder_state_inputs,
    [decoder_outputs]+decoder_states)

#Reverse-lookup taken index to decode sequences back to
#something readable
reverse_input_char_index=dict(
    (i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict(
    (i,char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    #Encode the input as state vector
    states_value=encoder_model.predict(input_seq)
    
    #Generate empty target sequence of length 1.
    target_seq=np.zeros((1,1,num_decoder_tokens))
    #Populate the first character of target sequence with the start character.
    target_seq[0,0,target_token_index["\t"]]=1.
    
    #sampling loop for a batch of sequences
    #(to simplify here we assume a batch of size 1).
    stop_condition=False
    decoded_sentence=""
    while not stop_condition:
        output_tokens,h,c= decoder_model.predict(
            [target_seq]+states_value)
        
        #sample a token
        sampled_token_index=np.argmax(output_tokens[0,-1,:])
        sampled_char=reverse_target_char_index[sampled_token_index]
        decoded_sentence+=sampled_char
        
        #Exit condition:either hit max length
        #or find stop character.
        if (sampled_char=="\n"or
           len(decoded_sentence)>max_decoder_seq_length):
            stop_condition=True
            
        #update the target sequence (of lenght 1).
        target_seq=np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index]=1.
        
        #update states
        states_value=[h,c]
        
    return decoded_sentence


for seq_index in range(100):
    #Take one sequence (part of the training set)
    #for trying out decoding.
    input_seq=encoder_input_data[seq_index:seq_index +1]
    decoded_sentence=decode_sequence(input_seq)
    print("-")
    print("Input Sentence:",input_texts[seq_index])
    print("Decoded Sentence:",decoded_sentence)  

-
Input Sentence: Go.
Decoded Sentence: Va !

-
Input Sentence: Run!
Decoded Sentence: Cours !

-
Input Sentence: Run!
Decoded Sentence: Cours !

-
Input Sentence: Wow!
Decoded Sentence: Ça alors !

-
Input Sentence: Fire!
Decoded Sentence: Au feu !

-
Input Sentence: Help!
Decoded Sentence: À l'aide !

-
Input Sentence: Jump.
Decoded Sentence: Saute.

-
Input Sentence: Stop!
Decoded Sentence: Arrête-toi !

-
Input Sentence: Stop!
Decoded Sentence: Arrête-toi !

-
Input Sentence: Stop!
Decoded Sentence: Arrête-toi !

-
Input Sentence: Wait!
Decoded Sentence: Attendez !

-
Input Sentence: Wait!
Decoded Sentence: Attendez !

-
Input Sentence: Go on.
Decoded Sentence: Poursuis.

-
Input Sentence: Go on.
Decoded Sentence: Poursuis.

-
Input Sentence: Go on.
Decoded Sentence: Poursuis.

-
Input Sentence: I see.
Decoded Sentence: Je comprends.

-
Input Sentence: I try.
Decoded Sentence: J'essaye.

-
Input Sentence: I won!
Decoded Sentence: Je l'ai emporté !

-
Input Sentence: I won!
Decoded 